In [139]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert

from datetime import datetime, timedelta

import pandas as pd
import numpy as np
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [140]:
# 컬럼 리스트 만들기
col_list = ['순위', '영화명', '개봉일', '매출액', '매출액점유율', 
            '매출액증감(전일대비)', '매출액증감율(전일대비) ','누적매출액 ', 
            '관객수 ', '관객수증감(전일대비) ', '관객수증감율(전일대비) ', 
            '누적관객수 ','스크린수 ', '상영횟수 ', '대표국적 ', '국적 ', 
            '제작사 ', '배급사 ', '등급 ', '장르 ','감독 ', '배우 ','기준일']

# 빈 데이터 프레임 만들기
df1 = pd.DataFrame(columns = col_list)


for file in glob(r"C:\workspace\City_Rock\최종\*.csv"):
    base = pd.read_csv(file)
    df1 = pd.concat([base, df1], axis=0)

In [141]:
df1.shape

(570807, 23)

In [142]:
new_col_list = ['순위', '영화명', '개봉일', '등급 ', '장르 ', '대표국적 ', 
                '국적 ', '제작사 ', '배급사 ','감독 ', '배우 ','기준일',
                '매출액','누적매출액 ','관객수 ','누적관객수 ','스크린수 ', 
                '상영횟수 ']
new_col_name = ['순위', '영화명', '개봉일', '등급', '장르', '대표국적', 
                '국적', '제작사', '배급사','감독', '배우 ','기준일',
                '매출액','누적매출액','관객수','누적관객수','스크린수', 
                '상영횟수']

In [143]:
df2 = df1[new_col_list].copy()

# 컬럼명 띄어쓰기 없애기
df2.columns = new_col_name

# 영화명 결측인 행 제거
df2 = df2[df2["영화명"].isnull()==False].copy()

df2.shape

(563804, 18)

In [144]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 563804 entries, 0 to 21966
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   순위      563804 non-null  object
 1   영화명     563804 non-null  object
 2   개봉일     494385 non-null  object
 3   등급      502071 non-null  object
 4   장르      557515 non-null  object
 5   대표국적    560576 non-null  object
 6   국적      560576 non-null  object
 7   제작사     222078 non-null  object
 8   배급사     482599 non-null  object
 9   감독      527125 non-null  object
 10  배우      484149 non-null  object
 11  기준일     563804 non-null  object
 12  매출액     563804 non-null  object
 13  누적매출액   563804 non-null  object
 14  관객수     563804 non-null  object
 15  누적관객수   563804 non-null  object
 16  스크린수    563804 non-null  object
 17  상영횟수    563804 non-null  object
dtypes: object(18)
memory usage: 81.7+ MB


In [145]:
# 컬럼 dtype 변경
df2["순위"] = df2["순위"].astype(np.int64)
df2["개봉일"] = pd.to_datetime(df2["개봉일"], format="%Y-%m-%d")
df2["기준일"] = pd.to_datetime(df2["기준일"], format="%Y-%m-%d")
df2["매출액"] = df2["매출액"].astype(np.int64)
df2["누적매출액"] = df2["누적매출액"].astype(np.int64)
df2["관객수"] = df2["관객수"].astype(np.int64)
df2["누적관객수"] = df2["누적관객수"].astype(np.int64)
df2["스크린수"] = df2["스크린수"].astype(np.int64)
df2["상영횟수"] = df2["상영횟수"].astype(np.int64)

In [146]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 563804 entries, 0 to 21966
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   순위      563804 non-null  int64         
 1   영화명     563804 non-null  object        
 2   개봉일     494385 non-null  datetime64[ns]
 3   등급      502071 non-null  object        
 4   장르      557515 non-null  object        
 5   대표국적    560576 non-null  object        
 6   국적      560576 non-null  object        
 7   제작사     222078 non-null  object        
 8   배급사     482599 non-null  object        
 9   감독      527125 non-null  object        
 10  배우      484149 non-null  object        
 11  기준일     563804 non-null  datetime64[ns]
 12  매출액     563804 non-null  int64         
 13  누적매출액   563804 non-null  int64         
 14  관객수     563804 non-null  int64         
 15  누적관객수   563804 non-null  int64         
 16  스크린수    563804 non-null  int64         
 17  상영횟수    563804 non-null  int64

In [147]:
# 기준일 기준 오름차순 정렬
df3 = df2.sort_values(by=["기준일"]).copy()
df3

,순위,영화명,개봉일,등급,장르,대표국적,국적,제작사,배급사,감독,배우,기준일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수
21966,1,동갑내기 과외하기,2003-02-07,12세관람가,코미디,한국,한국,코리아 엔터테인먼트,CJ ENM,김경형,"김하늘,권상우,공유,김지우,백일섭,김혜옥,김자옥,오승근,이성진,손영민,송병철,신지현...",2003-01-12,633500,633500,102,102,1,1
239,1,매트릭스3 레볼루션,2003-11-05,15세관람가,"SF,액션,범죄",미국,미국,NaN,워너브러더스 코리아(주),"릴리 워쇼스키,라나 워쇼스키","키아누 리브스,로렌스 피쉬번,캐리 앤 모스",2003-11-11,7676000,7676000,870,870,6,16
240,2,위대한 유산,2003-10-24,15세관람가,코미디,한국,한국,CJ ENM,CJ ENM,오상훈,"김선아,임창정,최무인,송은정,홍금봉,신성아,김대중,오유진,유순철,신이,모지은,김기천...",2003-11-11,1658000,1658000,177,177,1,3
241,3,황산벌,2003-10-17,15세관람가,코미디,한국,한국,(주)씨네월드,(주)씨네월드,이준익,"박중훈,정진영,오지명,김병철,김선아,김윤태,이문식,조대은,김만수,김현두,나수원,조철...",2003-11-11,1257000,1257000,116,116,1,3
242,4,아이덴티티,2003-10-31,15세관람가,"범죄,스릴러",미국,미국,콜럼비아트라이스타,콜럼비아트라이스타,제임스 맨골드,"존 쿠삭,레이 리오타",2003-11-11,821000,821000,102,102,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245965,29,코만체로스,NaT,NaN,"액션,서부극(웨스턴)",미국,미국,NaN,NaN,마이클 커티즈,"존 웨인,스튜어트 휘트먼",2022-12-31,420000,10290000,210,4948,1,4
245964,28,양자경의 더 모든 날 모든 순간,2022-11-23,15세이상관람가,"액션,코미디",미국,미국,NaN,워터홀컴퍼니(주),"다니엘 콴,다니엘 쉐이너트","양자경,스테파니 수,키 호이 콴,제이미 리 커티스",2022-12-31,2703789,291016371,276,27325,7,8
245963,26,캐리와 슈퍼콜라,2023-01-18,전체관람가,애니메이션,한국,한국,(주)캐리소프트,롯데컬처웍스(주)롯데엔터테인먼트,"오성윤,이춘백","최정윤,엄상현,이다은,원옥화,정주원,이창민,한상태,김지정,강나후,김신비,소경호",2022-12-31,3993000,3993000,295,295,1,1
245974,38,지난 해 마리앙바드에서,NaT,NaN,"드라마,미스터리,멜로/로맨스",프랑스,"프랑스,이탈리아",NaN,NaN,알랭 레네,"델핀 세이릭,사샤 피퇴프,조르지오 알베르타찌",2022-12-31,577000,3395000,82,582,1,1


In [155]:
# 개봉일 결측인 행 제거
# 데이터 원본자체의 결측
# 대부분 일반적인 영화가 아님 (젝스키스, 통합 등)
# 일반 영화 중 개봉일 결측인 경우 매출액이 상대적으로 많이 적어 제외

df4 = df3[df3["개봉일"].isnull()==False].copy()
df4.isnull().sum()

순위            0
영화명           0
개봉일           0
등급         7347
장르          571
대표국적        129
국적          129
제작사      278391
배급사       18449
감독        15626
배우        42582
기준일           0
매출액           0
누적매출액         0
관객수           0
누적관객수         0
스크린수          0
상영횟수          0
dtype: int64

In [156]:
df4.to_csv("(완)[KOBIS] 박스오피스_일별(2003~2022)_수정.csv", index=False)